<a href="https://colab.research.google.com/github/jmarcano101/data110/blob/main/spotify_songs_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.offline import iplot, plot
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/spotifysongs (1) (1).csv')

In [ ]:
df.head()

In [ ]:
df.info()

**There is no null vlues!**

In [ ]:
df.describe()

**Let's look into what each feature means:**
- **artist**: Name of the Artist.
- **song**: Name of the Track.
- **duration_ms**: Duration of the track in milliseconds.
- **explicit**: The lyrics or content of a song or a music video contain one or more of the criteria which could be considered offensive or unsuitable for children.
- **year**: Release Year of the track.
- **popularity**: The higher the value the more popular the song is.
- **danceability**: Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.
- **energy**: Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity.
- **key**: The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. If no key was detected, the value is -1.
- **loudness**: The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typically range between -60 and 0 db.
- **mode**: Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0.
- **speechiness**: Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks.
- **acousticness**: A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic.
- **instrumentalness**: Predicts whether a track contains no vocals. "Ooh" and "aah" sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly "vocal". The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.
- **liveness**: Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live.
- **valence**: A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).
- **tempo**: The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration.
- **genre**: Genre of the track.

## Analysis && Visualization

**Query Number 1: How many songs we have from each year?**

In [ ]:
songs_per_years = df['year'].value_counts().sort_index()

iplot(px.bar(songs_per_years,
            title='Number of songs per year',
            text_auto=True,
            labels=dict(index='year', value='number of songs'),
            color_discrete_sequence=['#1DB954']
            ).update_xaxes(type='category'))

**Query Number 2: All the artists in this list**

In [ ]:
print(f"There are about [{df['artist'].nunique()}] artists.")

In [ ]:
artists = df['artist'].value_counts()
artists.head()

**We have a total of 835, Let's explore the top**

In [ ]:
iplot(px.pie(values=[artists[:50].sum(), 2000-artists[:50].sum()],
            names=['top 50 artists', f'other{835-50} artists'],
            title='How many songs do the top 50 artists have?',
            color_discrete_sequence=['#4287f5', '#f54287']
            ).update_traces(textinfo='value+percent'))

**Query Number 3: No. of artists with only 1 song VS artists with more than 1**

In [ ]:
artists_with_one_song = artists.loc[lambda x: x==1]
iplot(px.pie(values=[len(artists_with_one_song), len(artists)-len(artists_with_one_song)],
            names=['1 song', '> 1 song'],
            title="Artists with 1 top hit VS Artitsts with > 1 top hit",
            color_discrete_sequence=['#4287f5', '#f54287']
            ).update_traces(textinfo='label+percent'))

**Query Number 4: Top 10 Artists with the average popularity of their songs**

In [ ]:
artist_df = df[['artist', 'popularity']].groupby('artist').mean().sort_values(by='artist')
artists = artists.sort_index()
artist_df['total songs'] = artists.values
artist_df.sort_values(by='total songs', ascending=False, inplace=True)
artist_df.reset_index(inplace=True)

iplot(px.scatter(artist_df[:10],
                x='artist',
                y='popularity',
                size='total songs',
                size_max=40,
                color='popularity',
                title='Top 10 artists VS Average popularity of their top hits',
                hover_name='total songs'
                ))

**Query Number 5: How has the average duration of songs changed through the years?**

In [ ]:
df['year'].value_counts().sort_values()

In [ ]:
# Function to convert duration in ms into min: sec
def ms_to_minsec(ms):
    sec = ms / 1000
    return f"{int(sec // 60)}:{int(sec % 60)}"

# Skip the years 1998 and 1999
durations = df[['duration_ms', 'year']].groupby('year').mean().reset_index().iloc[2:]

# Convert duration in ms into seconds
durations['duration_s'] = durations['duration_ms'] / 1000

# Apply ms_to_minsec function
durations['min:sec'] = durations['duration_ms'].apply(ms_to_minsec)

# Plot
iplot(px.line(durations,
             x='year',
             y='duration_s',
             title='average song duration over the years',
             text='min:sec'
             ).update_xaxes(type='category').update_traces(textposition='top right'))

**Query Number 6: Longest song VS Shortest song**

In [ ]:
asdy = df.loc[:, ['artist', 'song', 'duration_ms', 'year']]
asdy['min:sec'] = asdy['duration_ms'].apply(ms_to_minsec)

In [ ]:
# Longest song
asdy[asdy.duration_ms == asdy.duration_ms.max()]

In [ ]:
# Shortest song
asdy[asdy.duration_ms == asdy.duration_ms.min()]

**Query Number 7: Explicit VS Clean over the years**

In [ ]:
iplot(px.pie(df['explicit'].value_counts().reset_index(),
            values='explicit',
            names=['Clean', 'Explicit'],
            title='Explicit or not?',
            color_discrete_sequence=['cornflowerblue', 'crimson']
            ).update_traces(textinfo='label+percent'))

In [ ]:
year_explicit = df.groupby(['year', 'explicit']).size().unstack(fill_value=0).reset_index()
year_explicit.rename(columns={False: 'Clean', True: 'Explicit'}, inplace=True)

iplot(px.bar(year_explicit,
            y=['Clean', 'Explicit'],
            x='year',
            title='Explicit VS Clean distribution each year',
            color_discrete_sequence=['cornflowerblue', 'crimson']
            ).update_xaxes(type='category'))

**Query Number 8: Pitch Class Notation - Key**

- **In music, the key determines the set of notes and chords that are commonly used in a piece of music.**

In [ ]:
iplot(px.pie(df['key'].value_counts().reset_index(),
            names=r'C C♯/D♭ D E♭/D♯ E F F♯/G♭ G A♭/G♯ A B♭/A♯ B'.split(),
            values='key',
            color_discrete_sequence = px.colors.qualitative.Set3,
           title='Key Distribution'
          ).update_traces(textinfo='label+percent'))

**Query Number 9: Major and Minor of a track - mode**

- **In music theory, the mode refers to the overall tonal quality or mood of a piece of music, and it is typically categorized into two main modes: major and minor.**

In [ ]:
iplot(px.pie(df['mode'].value_counts().reset_index(),
       names=['major','minor'],
       values='mode',
       color_discrete_sequence = px.colors.qualitative.Pastel2,
       title='Major/Minor Distribution'
      ).update_traces(textinfo='label+percent'))

**Query Number 10: Major VS Minor in each Key**

In [ ]:
key_mode = df.groupby(['key', 'mode']).size().unstack(fill_value=0).reset_index()
key_names = r'C C♯/D♭ D E♭/D♯ E F F♯/G♭ G A♭/G♯ A B♭/A♯ B'.split()
key_mode.rename(columns={0:'minor',1:'major'}, inplace=True)
key_mode['key name'] = key_names

iplot(px.bar(key_mode,
            x='key name',
            y=['major', 'minor'],
            color_discrete_sequence=px.colors.qualitative.Pastel2,
           title='Major vs Minor per key'
          ))

**Query Number 11: Distribution**

In [ ]:
histogram_labels = ['popularity',
                    'danceability',
                    'energy',
                    'speechiness',
                    'loudness',
                    'acousticness',
                    'liveness',
                    'instrumentalness',
                    'valence',
                    'tempo'
                   ]

colors = px.colors.qualitative.Vivid
for i in range(len(histogram_labels)):
    fig = px.histogram(df,
                       histogram_labels[i],
                       title=f'{histogram_labels[i]} distribution in top hits',
                       color_discrete_sequence=[colors[i]]
                      )
    iplot(fig)

**Query Number 12: Genres**

In [ ]:
def split_genres(genre):
    g = genre.replace(" ", '').split(',')
    g = [t for t in g if t != 'set()']
    return g

def flatten(t):
    return [item for sublist in t for item in sublist]

def remove_duplicates(l):
    res = []
    l = [res.append(x) for x in l if x not in res]
    return res

all_genres = remove_duplicates(flatten(list(df['genre'].apply(split_genres))))

genre_count = {genre: 0 for genre in all_genres}

for song in list(df['genre']):
    for genre in split_genres(song):
        genre_count[genre]+=1


In [ ]:
iplot(px.pie(names=genre_count.keys(),
       values=[genre_count[key] for key in genre_count.keys()],
       title="Genre Distribution"
      ).update_traces(textinfo='label+percent'))